In [3]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [16]:
%pip install langchainhub

In [1]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
import getpass
import os

In [3]:
text = """Sdn     WHY IS THE BLOOD MOON RED?     Sdn     Sdn     Sdn     Sdn  Sdn   As light from the sun hits 
the earth it casts a shadow across space. A lunar eclipse occurs when the moon passes into the shadow. 
However a blood moon is when only the red part of visible light diffracts or bends around the horizon of our
planet washing the moon in a deep red tint."""

In [4]:
# Specify the file name
file_name = "output1234.txt"

# Open the file in write mode ('w' mode)
with open(file_name, 'w') as file:
    # Write the string to the file
    file.write(text)

print("String has been written to the file:", file_name)


String has been written to the file: output1234.txt


In [5]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("output1234.txt")
docs = loader.load()

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [7]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [8]:
splits

[Document(page_content='Sdn     WHY IS THE BLOOD MOON RED?     Sdn     Sdn     Sdn     Sdn  Sdn   As light from the sun hits \nthe earth it casts a shadow across space. A lunar eclipse occurs when the moon passes into the shadow. \nHowever a blood moon is when only the red part of visible light diffracts or bends around the horizon of our\nplanet washing the moon in a deep red tint.', metadata={'source': 'output1234.txt'})]

In [9]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [10]:
vectorstore

In [11]:
# prompt = hub.pull("rlm/rag-prompt")
# prompt

In [12]:
retriever=vectorstore.as_retriever()

In [13]:
from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
##YOU SHOULD ONLY ANSWER THE QUESTION IF IT IS IN GIVEN CONTEXT OTHERWISE DO NOT ANSWER##

{context}

Question: {question}
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [14]:
from langchain.chains import RetrievalQA
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs   # pass kwargs here
)

In [20]:
query = "who is sachin tendulkar"
qa.run(query)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


"I don't know."

In [16]:
# # LLM
# from langchain.chains import RetrievalQA


# # RetrievalQA
# qa_chain = RetrievalQA.from_chain_type(
#     llm,
#     retriever=vectorstore.as_retriever(),
#     chain_type_kwargs={"prompt": prompt}
# )
# question = "How many moons does saturn have?"
# result = qa_chain({"query": question})
# result["result"]

ValidationError: 1 validation error for LLMChain
prompt
  value is not a valid dict (type=type_error.dict)